In [15]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output
from create_dataframe import create_dataframe

In [16]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', None)

In [17]:
create_dataframe()

df = pd.read_csv('grants.csv')
df

,project_name,grant_name,year,organization,organization_short_name,lat,lon
0,Модель разрушения тонкого адгезионного слоя с упругопластическими свойствами на основе масштабируемого линейного параметра,Проведение инициативных исследований молодыми учеными,2022.0,"федеральное государственное бюджетное образовательное учреждение высшего образования ""Тульский государственный университет""","""Тульский государственный университет""",54.167370,37.585117
1,Новые подходы к цифровой обработке биомедицинских данных на основе параллельных вычислений и искусственных нейронных сетей,Проведение инициативных исследований молодыми учеными,2022.0,"федеральное государственное автономное образовательное учреждение высшего образования ""Северо-Кавказский федеральный университет""","""Северо-Кавказский федеральный университет""",45.042803,41.964381
2,Суперкомпьютерные технологии мониторинга и экологического проектирования состояния мелководных водоемов и морских систем с учетом разномасштабной турбулентности,Проведение инициативных исследований молодыми учеными,2022.0,"Федеральное государственное бюджетное образовательное учреждение высшего образования ""Ростовский государственный экономический университет (РИНХ)""","""Ростовский государственный экономический университет (РИНХ)""",48.932362,40.374866
3,Цифровая модель экспериментальной физической среды диагностики наноматериалов на основе синхротронного излучения для тренировки агентов глубокого обучения с подкреплением,Проведение инициативных исследований молодыми учеными,2022.0,"федеральное государственное автономное образовательное учреждение высшего образования ""Южный федеральный университет""","""Южный федеральный университет""",47.216413,39.631551
4,"Распространение электромагнитных одночастотных связанных волн в плоских волноведущих структурах, заполненных нелинейной средой",Проведение инициативных исследований молодыми учеными,2022.0,"федеральное государственное бюджетное образовательное учреждение высшего образования ""Пензенский государственный университет""","""Пензенский государственный университет""",53.181998,45.002486
...,...,...,...,...,...,...,...
8753,Разработка и исследование методов повышения надежности привязных высотных беспилотных телекоммуникационных платформ нового поколения,Проведение фундаментальных научных исследований и поисковых научных исследований международными научными коллективами,NaN,федеральное государственное бюджетное учреждение науки Институт проблем управления им. В. А. Трапезникова Российской академии наук,Институт проблем управления им. В. А. Трапезникова Российской академии наук,55.651365,37.537632
8754,Разработка легких наноструктурированных функционально-градиентных материалов для высокопрочных применений с помощью методов гибридной асимметричной прокатки и инкрементальной формовки,Проведение фундаментальных научных исследований и поисковых научных исследований международными научными коллективами,NaN,"федеральное государственное бюджетное образовательное учреждение высшего образования ""Магнитогорский государственный технический университет им. Г.И. Носова""","""Магнитогорский государственный технический университет им. Г.И. Носова""",53.421976,58.981235
8755,Летательный аппарат дальнего действия для исследования Марса,Проведение фундаментальных научных исследований и поисковых научных исследований международными научными коллективами,NaN,"федеральное государственное бюджетное образовательное учреждение высшего образования ""Московский авиационный институт (национальный исследовательский университет)""","""Московский авиационный институт (национальный исследовательский университет)""",55.740159,37.406807
8756,Технологические решения для максимальной интеграции возобновляемых источников в электрические сети,Проведение фундаментальных научных исследований и поисковых научных исследований международными научными коллективами,NaN,Автономная некоммерческая образовательная организация высшего образования «Сколковский институт науки и технологий»,«Сколковск

In [18]:
px.set_mapbox_access_token(open(".mapbox_token").read()[:-1])

In [19]:
organizations = df \
    .groupby(['organization_short_name', 'year'], as_index=False, dropna=False) \
    .agg({'project_name': 'count', 'lat': 'max', 'lon': 'max'}) \
    .rename(columns={"project_name" : 'number_of_projects'})

In [20]:
years = df.year.sort_values().unique()

fig = px.scatter_mapbox(organizations[organizations.year==years[0]],
                        lat='lat',
                        lon='lon',
                        hover_name="organization_short_name",
                        size='number_of_projects',
                        center=dict(lat=60, lon=50),
                        zoom=3)



app = JupyterDash(__name__)


app.layout = html.Div([
    html.P("Select a year:"),
    dcc.RadioItems(
        id='year', 
        options=years,
        value=years[0],
        inline=True
    ),
    dcc.Graph(figure=fig, id='graph'),
])

@app.callback(
    Output("graph", "figure"), 
    Input("year", "value"))
def display_scatter_mapbox(year):
    if year:
        organization_by_year = organizations[organizations.year==year]
    else:
        organization_by_year = organizations[organizations.year.isna()]
    fig = px.scatter_mapbox(organization_by_year,
                        lat='lat',
                        lon='lon',
                        hover_name="organization_short_name",
                        size='number_of_projects',
                        center=dict(lat=60, lon=50),
                        zoom=3)
    return fig



app.run_server(mode='inline', port=8090, dev_tools_ui=True,
              dev_tools_hot_reload=True, threaded=True)

Dash is running on http://127.0.0.1:8090/



[INFO] Dash is running on http://127.0.0.1:8090/



In [21]:
years = df.year.sort_values().unique()

fig = px.density_mapbox(organizations[organizations.year==years[0]], 
                        lat='lat', 
                        lon='lon', 
                        z='number_of_projects', 
                        radius=10,
                        center=dict(lat=60, lon=50), 
                        zoom=2)



app = JupyterDash(__name__)


app.layout = html.Div([
    dcc.RadioItems(
        id='year', 
        options=years,
        value=years[0],
        inline=True
    ),
    dcc.Graph(figure=fig, id='graph'),
])

@app.callback(
    Output("graph", "figure"), 
    Input("year", "value"))
def display_density_mapbox(year):
    if year:
        organization_by_year = organizations[organizations.year==year]
    else:
        organization_by_year = organizations[organizations.year.isna()]
    fig = px.density_mapbox(organization_by_year, 
                        lat='lat', 
                        lon='lon', 
                        z='number_of_projects', 
                        radius=10,
                        center=dict(lat=60, lon=50), 
                        zoom=2)
    return fig



app.run_server(mode='inline', port=8091, dev_tools_ui=True,
              dev_tools_hot_reload=True, threaded=True)

Dash is running on http://127.0.0.1:8091/



[INFO] Dash is running on http://127.0.0.1:8091/



In [22]:
without_location = df[df.lat.isna()] \
    .groupby('organization_short_name', as_index=False) \
    .agg({'project_name': 'count'}) \
    .sort_values(by='project_name', ascending=False) \
    .reset_index(drop=True)
without_location

,organization_short_name,project_name
0,"""Федеральный исследовательский центр Институт прикладной физики Российской академии наук""",70
1,"Федеральный исследовательский центр ""Институт общей физики им. А.М. Прохорова Российской академии наук""",54
2,Институт органической химии им. Н.Д. Зелинского Российской академии наук,47
3,Физико-технический институт им. А.Ф. Иоффе Российской академии наук,47
4,Институт проблем экологии и эволюции им. А.Н. Северцова Российской академии наук,38
...,...,...
112,"""Санкт-Петербургский государственный университет гражданской авиации имени Главного маршала авиации А.А. Новикова""",1
113,"""ПСК ЭНЕРГОМОНТАЖ""",1
114,"""Новые спинтронные технологии""",1
115,"""Национальный медицинский исследовательский центр трансплантологии и искусственных органов имени академика В.И. Шумакова"" Министерства здравоохранения Российской Федерации",1
